In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import logsumexp
from scipy.stats import norm
import random

# Set some defaults
plt.rc("axes.spines", top=False, right=False)

sns.set_theme(context="paper", font_scale=1.2)
sns.set_style("ticks")

# Import python library/function for function optimization
import scipy.optimize

%config InlineBackend.figure_format = "retina"

## The Plan

### Parameter and model recovery (Can I do both at same time?)
1) Simulate data using best-fit parameters from each model (or samples from within the range of best-fits). For example, 100 simulations of each model within range of best-fit parameters.
2) Re-estimate parameters.
3) Model comparison with AIC/BIC. 
4) Correlate parameters used for simulation with recovered parameters.


In [ ]:
for i in range(len(subjects)):
    # Unpack parameters
    pea_mle = params.loc[params["model"] == "pea", "theta"].values[0]

    # Simulate
    # PEA model
    _, xhat_pea = pea(
        pea_mle[0], 
        pea_mle[1], 
        sigma_motor, len(sample_subj), 
        sample_subj["fbi"], 
        sample_subj["rotation"],
        fit=False
    )

    # Fit simulated data
    # PEA model params: sigma_comb, B
        bounds = ((0.5, 25), (0, 1))
        tmp_pea = scipy.optimize.minimize(
            fun=nll_pea, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1])])
        )
    
    # Model selection
    bic.extend([calc_bic(pea_results.fun * -1, len(pea_results.x), len(subj))])
    aic.extend...